# PIX2PIX EXAMPLE (TENSORFLOW 2.0) (UNDER CONSTRUCTION)

This is an example of the Pix2Pix architecture. In this example we will have to datasets: one dataset having the input flower pictures and the second dataset having the same input flower pictures but with random noise.
U-UNET = SKIP CONNECTIONS!
Using skip connections in order not to have a very small bottleneck!.
Check what is patchGAN

In [ ]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np


class ImageTools(object):

    def __init__(self, images):
        self.__images = images #List with pair of images [input_image, target_image] (it can have more images)

    def __str__(self):
        return "Images object of length {}".format(len(self.__images))
    
    def __len__(self):
        return len(self.__images)

    def normalize(self):
        """ This function normalizes the images of rank (0,255) to (-1, 1)"""
        
        normalized_images = []
        for image in self.__images:
                normalized_images.append((image / 127.5)-1)
        return normalized_images

    def left_right_random_flipper(self):
        """randomly flipping the image"""
        flipped = [tf.image.flip_left_right(image) for image in self.__images if tf.random.uniform(()) > 0.5]
        return flipped
    
    def up_down_random_flipper(self):
        flipped = [tf.image.flip_up_down(image) for image in self.__images if tf.random.uniform(()) > 0.5]
        return flipped
    
    def random_jitter(self):

        """
        The image is cropped randomly. The first size dimension is the number of pictures (two stacked pictures)
        The last dimension size is the number of channels, in this case 3 (RGB)
        """
        stacked_images = tf.stack(self.__images, axis=0)
        cropped_images = tf.image.random_crop(stacked_images, size=[len(self.__images), IMG_HEIGTH, IMG_WIDTH, 3]) #Putting images together in order to perform the same actions


class NetworkUtilities(object):
    
    def __init__(self, LAMBDA=100):
        
        self.__LAMBDA = LAMBDA
    
    def _encoder(self, filters, apply_batchnorm=True): #encoder
    
        encoder_block = Sequential()
        initializer = tf.random_normal_initializer(0, 0.02)

        #Convolutional layer, use bias as long as we are not applying batch normalization since batch normalization add
        # a bias by default!!
        encoder_block.add(Conv2D(filters, kernel_size=4, strides=2, padding="same", kernel_initializer=initializer, use_bias=not apply_batchnorm))

        #Batch Normalization
        if apply_batchnorm:
            encoder_block.add(batchNormalization())
        encoder_block.add(LeakyReLu)

        return encoder_block

    
    def _decoder(self, filters, apply_dropout=False): #Decoder
    
        decoder_block = Sequential()

        initializer = tf.random_normal_initializer(0, 0.02)

        #Convolutional layer, use bias as long as we are not applying batch normalization since batch normalization add
        # a bias by default!!
        decoder_block.add(Conv2DTranspose(filters, kernel_size=4, strides=2, padding="same", kernel_initializer=initializer, use_bias=False))

        #Batch Normalization
        decoder_block.add(batchNormalization())

        #Dropout later (regulariazation technique)
        if apply_dropout:
            decoder_block.add(Dropout(0.5))

        #Activation layer
        decoder_block.add(ReLu) #Relu as specified in the paper

        return decoder_block
    
    @property
    def loss_object(self):
        
        return tf.keras.losses.BinaryCrossentropy(from_logits=True) #From_logits=True we pass the images thru sigmoid function
    
    def _generator_loss(self, disc_generated_output, gen_output, target):
    
        gan_loss = self.loss_object(tf.ones(disc_generated_output), disc_generated_output)
        l1_loss = tf.reduce_mean(tf.abs(target - gen_output))
        total_gen_loss = gan_loss + (self.__LAMBDA * l1_loss)
        
        return total_gen_loss
    
    def _discriminator_loss(self, disc_real_output, disc_generated_output):

        real_loss = self.loss_object(tf.ones(disc_real_output), disc_real_output) #The Ones matrix mean that they are real!
        generated_loss = self.loss_object(tf.zeros_like(disc_generated_output), disc_generated_output) #Zeros means false
        total_disc_loss = real_loss + generated_loss

        return total_disc_loss
    
    @property
    def _optimizer(self):
        
        return tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
        
        
class NetworkArchitecture(NetworkUtilities):
    
    def __init__(self):
        super().__init__()
        
    def generator(self):
    
        inputs = tf.keras.layers.Input(shape=[None, None, 3]) #Initializing the shape [height, width, channels]
        down_stack = [
            self._encoder(64, apply_batchnorm=False), #(bs, 128, 128, 64) bs = batch_size, reduced to the half
            self._encoder(128), #(bs, 64, 64, 128) the channels increase due to the number of filters used!
            self._encoder(256), #(bs, 32, 32, 256)
            self._encoder(512),
            self._encoder(512),
            self._encoder(512),
            self._encoder(512),
            self._encoder(512), #(bs, 1, 1, 512)
        ]

        up_stack = [
            self._decoder(512, apply_dropout=True),
            self._decoder(512, apply_dropout=True),
            self._decoder(512, apply_dropout=True),
            self._decoder(512),
            self._decoder(256),
            self._decoder(128),
            self._decoder(64),
        ]
        
        initializer = tf.random_normal_initializer(0, 0.02) #For generating random weights

        final_layer = Conv2DTranspose(filters=3, 
                                  kernel_size=4,
                                  strides=2,
                                  padding="same",
                                  kernel_initializer=initializer,
                                  activation="tanh" #Since we are using normalized images from -1 to 1
                                  ) #filters define the channels of the final image, therefore filters=3

        x = inputs
        skip_connections = [] #List where we will add different elemnts
        concat = Concatenate()
        for down in down_stack:
            x = down(x)
            skip_connections.append(x)

        skip_connections = reversed(skip_connections[:-1]) 

        for up, skip in zip(up_stack, skip_connections):
            x = up(x)
            x = concat([x, skip_connections])
            final = final_layer(x)

            return Model(inputs=inputs, outputs=final) #We have to construct the model object
        
        
    def discriminator(self):
    
        ini = Input(shape=[None, None, 3], name="input_img")
        gen = Input(shape=[None, None, 3], name="gener_img")
        con = concatenate([ini, gen]) #concatenating the two images generated and input

        initializer = tf.random_normal_initializer(0, 0.02)
        down1 = downsample(64, apply_batchnorm=False)(con)
        down2 = downsample(128)(down1)
        down3 = downsample(256)(down2)
        down4 = downsample(512)(down3)

        last = tf.keras.layers.Conv2D(filters=1,
                                     kernel_size=4,
                                     strides=1,
                                     kernel_initializer=initializer,
                                     padding="same")(down4) #Only one filter since we are defining for each area of the image
                                     #If it is the image or not so only one channel (PATCHGAN!)

        return tf.keras.Model(inputs=[ini, gen], outputs=last)
        